# Capstone: The Battle of the Neighborhoods

## Introduction: Business Problem
In this project we will try to find an optimal location for a shopping mall. Specifically, this report will be targeted to stakeholders interested in opening a shopping mall  in Auckland, New Zealand. Since there are lots of malls in Auckland we will try to detect locations that are not already crowded with malls. We are also particularly interested in areas with no shopping malls in vicinity. We would also prefer locations as close to city center as possible, assuming that first two conditions are met. We will use our data science powers to generate a few most promising neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data
Based on definition of our problem, factors that will influence our decission are:
* number of existing malls in the neighborhood
* number of and distance to shopping malls in the neighborhood, if any
* distance of neighborhood from city center
* We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using Foursquare API geocoding
* number of shopping malls and their type and location in every neighborhood will be obtained using Foursquare API
* coordinate of Auckland center will be obtained using Foursquare API geocoding

### Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

In [2]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

# All requested packages already installed.



In [3]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [4]:
import geocoder

In [5]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [6]:
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

In [7]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [8]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [9]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [10]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



### Scrap data into a dataframe

In [11]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Auckland").text

In [12]:
soup = BeautifulSoup(data, 'html.parser')

In [13]:
neighborhoodList = []

In [14]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [15]:
# create a new DataFrame from the list
ak_df = pd.DataFrame({"Neighborhood": neighborhoodList})

ak_df.head()

,Neighborhood
0,List of suburbs of Auckland
1,Airport Oaks
2,"Albany, New Zealand"
3,"Alfriston, New Zealand"
4,Algies Bay


In [16]:
#print the number of rows
ak_df.shape

(200, 1)

### Get the Coordinates

In [19]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Auckland, New Zealand'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [20]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in ak_df["Neighborhood"].tolist() ]

In [21]:
coords

[[-36.70918287099994, 174.73309179600005],
 [-36.79359048580961, 174.61801235305123],
 [-36.72915999999998, 174.70893000000012],
 [-37.010493372999974, 174.9408516090001],
 [-36.84839904099994, 174.76438785300002],
 [-36.86301999999995, 174.74858000000006],
 [-36.847859693114465, 174.73708327694862],
 [-36.59818999999993, 174.81019000000003],
 [-36.84839904099994, 174.76438785300002],
 [-36.84839904099994, 174.76438785300002],
 [-36.89044821899995, 174.6870174820001],
 [-36.88819999999998, 174.7401900000001],
 [-36.80819999999994, 174.77019000000007],
 [-36.77452705999997, 174.7090190230001],
 [-36.69486999999998, 174.75065000000006],
 [-36.80523647199993, 174.788186794],
 [-36.797416829999975, 174.70436862300005],
 [-36.80892881099993, 174.72194536900008],
 [-36.92819999999995, 174.70019000000002],
 [-36.90945304999997, 174.9170680730001],
 [-36.98755915999993, 174.9893839770001],
 [-36.71926010899995, 174.73666409700002],
 [-36.858199999999954, 174.90019000000007],
 [-36.749577067999

In [22]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [23]:
# merge the coordinates into the original dataframe
ak_df['Latitude'] = df_coords['Latitude']
ak_df['Longitude'] = df_coords['Longitude']

In [24]:
# check the neighborhoods and the coordinates
print(ak_df.shape)
ak_df

(200, 3)


,Neighborhood,Latitude,Longitude
0,List of suburbs of Auckland,-36.709183,174.733092
1,Airport Oaks,-36.793590,174.618012
2,"Albany, New Zealand",-36.729160,174.708930
3,"Alfriston, New Zealand",-37.010493,174.940852
4,Algies Bay,-36.848399,174.764388
5,"Arch Hill, New Zealand",-36.863020,174.748580
6,"Ardmore, New Zealand",-36.847860,174.737083
7,Army Bay,-36.598190,174.810190
8,Auckland CBD,-36.848399,174.764388
9,Auckland waterfront,-36.848399,174.764388


In [25]:
# save the DataFrame as CSV file
ak_df.to_csv("ak_df.csv", index=False)

### Create Map of Auckland with Neighborhoods superimposed

In [26]:
# get the coordinates of Kuala Lumpur
address = 'Auckland, New Zealand'

geolocator = Nominatim(user_agent="klcara@gmail.com")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Auckland, New Zealand {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Auckland, New Zealand -36.852095, 174.7631803.


In [27]:
# create map of Toronto using latitude and longitude values
map_ak = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(ak_df['Latitude'], ak_df['Longitude'], ak_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_ak)  
    
map_ak

In [28]:
# save the map as HTML file
map_ak.save('map_ak.html')

### Use Forsquare API to explore the neighborhoods

In [29]:
# define Foursquare Credentials and Version
CLIENT_ID = 'FSAG5IITB5052FZ15SDSD4HXM2PXG2CMTC1RSNJ1RN13CM1G' # your Foursquare ID
CLIENT_SECRET = 'ICOL1Q3LQZYVHXXDPUZWOJQ2UH3MG5CDLM2VE4XYSBUM32CZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FSAG5IITB5052FZ15SDSD4HXM2PXG2CMTC1RSNJ1RN13CM1G
CLIENT_SECRET:ICOL1Q3LQZYVHXXDPUZWOJQ2UH3MG5CDLM2VE4XYSBUM32CZ


In [30]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(ak_df['Latitude'], ak_df['Longitude'], ak_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [31]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(7943, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,List of suburbs of Auckland,-36.709183,174.733092,Sherwood Park,-36.717437,174.735803,Park
1,List of suburbs of Auckland,-36.709183,174.733092,Deep Creek Brews & Eats,-36.713687,174.748776,Brewery
2,List of suburbs of Auckland,-36.709183,174.733092,Browns Bay Beach,-36.714291,174.750037,Beach
3,List of suburbs of Auckland,-36.709183,174.733092,La Tropezienne French Café,-36.713914,174.747718,Café
4,List of suburbs of Auckland,-36.709183,174.733092,Hatz Donburi,-36.713914,174.747985,Japanese Restaurant


In [32]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Airport Oaks,8,8,8,8,8,8
"Albany, New Zealand",64,64,64,64,64,64
"Alfriston, New Zealand",2,2,2,2,2,2
Algies Bay,100,100,100,100,100,100
"Arch Hill, New Zealand",100,100,100,100,100,100
"Ardmore, New Zealand",100,100,100,100,100,100
Army Bay,6,6,6,6,6,6
Auckland CBD,100,100,100,100,100,100
Auckland waterfront,100,100,100,100,100,100


In [33]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 247 uniques categories.


In [34]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Park', 'Brewery', 'Beach', 'Café', 'Japanese Restaurant',
       'Neighborhood', 'Fish & Chips Shop', 'Bistro', 'Pizza Place',
       'Indian Restaurant', 'Noodle House', 'Coffee Shop',
       'Ice Cream Shop', 'Supermarket', 'Grocery Store', 'Restaurant',
       'Asian Restaurant', 'Liquor Store', 'Food', 'Pharmacy',
       'Fruit & Vegetable Store', 'Playground', 'Gym', 'Hobby Shop',
       'Golf Course', 'Airport', 'Farmers Market', 'Department Store',
       'Bar', 'Stadium', 'Movie Theater', 'Shopping Mall',
       'Sushi Restaurant', 'Burger Joint', 'Multiplex', 'Bubble Tea Shop',
       'Portuguese Restaurant', 'Fast Food Restaurant',
       'Electronics Store', 'Furniture / Home Store',
       'American Restaurant', 'Juice Bar', 'Seafood Restaurant',
       'Garden Center', 'Sporting Goods Shop', 'Mexican Restaurant',
       "Women's Store", 'Arts & Crafts Store', 'Clothing Store',
       'Optical Shop'], dtype=object)

In [35]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### Analyze each neighborhood

In [36]:
# one hot encoding
ak_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ak_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ak_onehot.columns[-1]] + list(ak_onehot.columns[:-1])
ak_onehot = ak_onehot[fixed_columns]

print(ak_onehot.shape)
ak_onehot.head()

(7943, 248)


,Neighborhoods,African Restaurant,Airport,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bay,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Station,Business Service,Café,Cajun / Creole Restaurant,Campground,Candy Store,Child Care Service,Chinese Restaurant,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Dutch Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Hill,History Museum,Hobby Shop,Hockey Field,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Motel,Mountain,Movie Theater,Moving Target,Multiplex,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paintball Field,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rest Area,Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Squash Court,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Summer Camp,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Track Stadium,Trail,Train Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Waterfront,Wine Bar,Wine Shop,Winery,Women's Store,Yakitori Restaurant,Yoga Studio,Zoo
0,List of suburbs of Auckland,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [37]:
ak_grouped = ak_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(ak_grouped.shape)
ak_grouped

(196, 248)


,Neighborhoods,African Restaurant,Airport,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bay,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Station,Business Service,Café,Cajun / Creole Restaurant,Campground,Candy Store,Child Care Service,Chinese Restaurant,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Dutch Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Hill,History Museum,Hobby Shop,Hockey Field,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Motel,Mountain,Movie Theater,Moving Target,Multiplex,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paintball Field,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rest Area,Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Squash Court,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Summer Camp,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Track Stadium,Trail,Train Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Waterfront,Wine Bar,Wine Shop,Winery,Women's Store,Yakitori Restaurant,Yoga Studio,Zoo
0,Airport Oaks,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.375000,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.00,

In [38]:
len(ak_grouped[ak_grouped["Shopping Mall"] > 0])

92

### Create a new dataframe for shopping mall data

In [39]:
ak_mall = ak_grouped[["Neighborhoods","Shopping Mall"]]

In [40]:
ak_mall.head()

,Neighborhoods,Shopping Mall
0,Airport Oaks,0.00000
1,"Albany, New Zealand",0.03125
2,"Alfriston, New Zealand",0.00000
3,Algies Bay,0.02000
4,"Arch Hill, New Zealand",0.02000


### Cluster neighborhoods (k-means)

In [41]:
aclusters = 3

ak_clustering = ak_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=aclusters, random_state=0).fit(ak_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 2, 2, 2, 0, 2, 2, 2], dtype=int32)

In [42]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
ak_merged = ak_mall.copy()

# add clustering labels
ak_merged["Cluster Labels"] = kmeans.labels_

In [43]:
ak_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
ak_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Airport Oaks,0.00000,0
1,"Albany, New Zealand",0.03125,2
2,"Alfriston, New Zealand",0.00000,0
3,Algies Bay,0.02000,2
4,"Arch Hill, New Zealand",0.02000,2


In [44]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ak_merged = ak_merged.join(ak_df.set_index("Neighborhood"), on="Neighborhood")

print(ak_merged.shape)
ak_merged.head() # check the last columns!

(196, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Airport Oaks,0.00000,0,-36.793590,174.618012
1,"Albany, New Zealand",0.03125,2,-36.729160,174.708930
2,"Alfriston, New Zealand",0.00000,0,-37.010493,174.940852
3,Algies Bay,0.02000,2,-36.848399,174.764388
4,"Arch Hill, New Zealand",0.02000,2,-36.863020,174.748580


In [45]:
# sort the results by Cluster Labels
print(ak_merged.shape)
ak_merged.sort_values(["Cluster Labels"], inplace=True)
ak_merged

(196, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Airport Oaks,0.000000,0,-36.793590,174.618012
105,Mellons Bay,0.000000,0,-36.878200,174.920190
106,Middlemore,0.000000,0,-36.966209,174.844730
107,"Milford, New Zealand",0.012658,0,-36.772850,174.765940
112,Mount Eden,0.010000,0,-36.883602,174.754237
115,Murrays Bay,0.000000,0,-36.731652,174.746350
116,Māngere,0.000000,0,-36.980840,174.781594
104,Mechanics Bay,0.000000,0,-36.838200,174.780190
117,"Māngere Bridge, New Zealand",0.000000,0,-36.942180,174.787460
119,"Narrow Neck, New Zealand",0.000000,0,-36.815421,174.799342


In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(aclusters)
ys = [i+x+(i*x)**2 for i in range(aclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ak_merged['Latitude'], ak_merged['Longitude'], ak_merged['Neighborhood'], ak_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [47]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### Examine Clusters

Cluster 0

In [48]:
ak_merged.loc[ak_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Airport Oaks,0.000000,0,-36.793590,174.618012
105,Mellons Bay,0.000000,0,-36.878200,174.920190
106,Middlemore,0.000000,0,-36.966209,174.844730
107,"Milford, New Zealand",0.012658,0,-36.772850,174.765940
112,Mount Eden,0.010000,0,-36.883602,174.754237
115,Murrays Bay,0.000000,0,-36.731652,174.746350
116,Māngere,0.000000,0,-36.980840,174.781594
104,Mechanics Bay,0.000000,0,-36.838200,174.780190
117,"Māngere Bridge, New Zealand",0.000000,0,-36.942180,174.787460
119,"Narrow Neck, New Zealand",0.000000,0,-36.815421,174.799342


Cluster 1

In [49]:
ak_merged.loc[ak_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
78,"Howick, New Zealand",0.052632,1,-36.893730,174.931540
137,Pakuranga,0.066667,1,-36.914538,174.872924
129,Onehunga,0.057143,1,-36.920599,174.790655
187,"Torbay, New Zealand",0.090909,1,-36.694870,174.750650
13,"Beach Haven, New Zealand",0.090909,1,-36.694870,174.750650
194,Ōtara,0.076923,1,-36.962170,174.874850
12,"Bayview, New Zealand",0.083333,1,-36.774527,174.709019
149,"Red Beach, New Zealand",0.090909,1,-36.694870,174.750650
103,"Meadowbank, New Zealand",0.058824,1,-36.869501,174.829573
91,"Long Bay, New Zealand",0.090909,1,-36.684005,174.737186


Cluster 2

In [50]:
ak_merged.loc[ak_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
182,Te Papapa,0.033333,2,-36.918200,174.800190
157,"Royal Oak, New Zealand",0.044776,2,-36.910653,174.772330
181,Te Atatū South,0.030303,2,-36.869558,174.647331
8,Auckland waterfront,0.020000,2,-36.848399,174.764388
171,Sunnyhills,0.046512,2,-36.901522,174.885906
154,"Rosehill, New Zealand",0.020000,2,-36.848399,174.764388
153,"Rosedale, Auckland",0.021277,2,-36.746500,174.715221
28,Clover Park,0.026316,2,-36.982834,174.892208
1,"Albany, New Zealand",0.031250,2,-36.729160,174.708930
151,Remuera,0.020833,2,-36.877404,174.805492


### Observations:
Most of the shopping malls are centered in the central part of Auckland, with the most shopping malls being in cluster 1 and a moderate number of shopping malls in cluster 2. Cluster 0 has a very low concentration of shopping malls in the neighborhoods. This presents an opportunity for developers to open shopping malls in this area because of the low competition. Clusters 1 and 2 show a much greater concentration of shopping malls in neighboorhoods closest to the city center, therefore, we recommend that developers take advantage of the low level of competition in cluster 0 and open a shopping mall in the suburbs. This will provide a lower level of competition and likely a higher level of success of a new shopping mall development in this area. 